Considering the studies realized in the "MoA data analysyis" notebook, lets try some new variables.

In [7]:
import gc
import time
import pandas as pd
import numpy as np
import random
import re
import pickle
# from sklearn.cluster import Birch
# from sklearn.cluster import KMeans
# from sklearn.cluster import MiniBatchKMeans
# from sklearn.cluster import MeanShift
# from sklearn.mixture import GaussianMixture
# from sklearn.mixture import BayesianGaussianMixture
# from sklearn.cluster import SpectralClustering    # Takes too long
# from sklearn.cluster import DBSCAN                # Did not work
# import hdbscan                                    # Does not have a predict only feature
# from sklearn.decomposition import PCA
# import seaborn as sns
import matplotlib.pyplot as plt
# import plotly.express as px
# import plotly.graph_objects as go
%matplotlib inline

# Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable), and you're done!
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss


import torch
import torch.nn as nn
import torch.nn.functional as F

#Path to save the output
path_holdout = 'C:\\Users\\maxwi\\Python\\Kaggle\\Mechanism of action\\results 5 - FNN new variables\\'

In [3]:
#load the Data
train_features = pd.read_csv('train_features.csv') 
train_targets_scored = pd.read_csv('train_targets_scored.csv')
test_features = pd.read_csv('test_features.csv')

In [4]:
#Function to reduce memory usage.
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train_features = reduce_mem_usage(train_features)  
train_targets_scored = reduce_mem_usage(train_targets_scored)
test_features = reduce_mem_usage(test_features)

Mem. usage decreased to 40.18 Mb (74.8% reduction)
Mem. usage decreased to  4.86 Mb (87.1% reduction)
Mem. usage decreased to  6.72 Mb (74.8% reduction)


In [ ]:
def loss_train_validation(db, rotation_x, save_to):
    '''
    loss_train_validation(db = log_loss_hp, rotation_x = 0, save_to = save_file_final)
    '''
    #Plot train and validation log loss
    fig, ax = plt.subplots(figsize=(15, 10))
    #Train log loss:
    x = db['parameter_index']
    y = db['log_loss_train']

#     ax.xaxis.set_ticks(x)
    ax.plot(x, y, color='black', zorder=1, linewidth=0.7, linestyle = 'solid')
    # add two layers of points to create an illusion of a discontinuous line. "zorder" specifies plotting order
    ax.scatter(x, y, s = 64, color = 'white', zorder = 2)
    ax.scatter(x, y, s = 8, color = 'black', zorder = 3)
    ax.text(x.iloc[-1] + 0.001, y.iloc[-1], 'Train', va = 'center')

    #Validation log loss:
#     x = log_loss_hp['parameter']
    y = db['log_loss_validation']

#     ax.xaxis.set_ticks(x)
    ax.plot(x, y, color='black', zorder=1, linewidth=0.7, linestyle = 'dashed')
    # add two layers of points to create an illusion of a discontinuous line. "zorder" specifies plotting order
    ax.scatter(x, y, s=64, color='white', zorder=2)
    ax.scatter(x, y, s=8, color='black', zorder=3)
    ax.text(x.iloc[-1] + 0.001, y.iloc[-1], 'Validation', va = 'center')

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    plt.xticks(rotation = rotation_x)

    print()
    print('Smallest log loss:', round(db['log_loss_validation'].min(), 4))
    print('Smallest log loss parameter:', db[np.in1d(db['log_loss_validation'], db['log_loss_validation'].min())]['parameter'].values[0])
    log_loss_hp.to_csv(save_to, index = False)

In [5]:
def print_time_to_end(progress_count, step_size, already_read, full_db_size):
    '''
    progress_check = print_time_to_end(progress_count = progress_check, step_size = progress_mult
                            , already_read = progress, full_db_size = len(train_y.columns)*len(hyperparameter))
    '''
#     progress += 1   #Changed to satisfy both situation if loading or not a saved progress.
    if already_read == progress_count * step_size:
        print()
        progress_per = round(already_read / full_db_size, 4)
        print(progress_per)
        
        end = time.time()
        elapsed = int(round(end - start, 0))
        total_run_time =  int(round(elapsed / (progress_per), 0))
        time_to_finish = int(round(elapsed / (progress_per), 0)) - elapsed
        print('Elapsed: {:02d}:{:02d}:{:02d}'.format(elapsed // 3600, (elapsed % 3600 // 60), elapsed % 60))
        print('Total run time: {:02d}:{:02d}:{:02d}'.format(total_run_time // 3600, (total_run_time % 3600 // 60), total_run_time % 60))
        print('Time to finish: {:02d}:{:02d}:{:02d}'.format(time_to_finish // 3600, (time_to_finish % 3600 // 60), time_to_finish % 60))
        print()
        return 1 #Increase progress_count
    
    return 0 #Don't increase progress_count

In [8]:
SEED = 1

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [25]:
'''
Lets separate train_features and train_targets_scored into train_set and validation_set.
This is important to avoid overfitting.
'''
train_features_set, validation_features_set = train_test_split(train_features, test_size = 0.2, random_state = 1).copy()

train_labels_set = train_targets_scored.iloc[train_features_set.index].copy()
validation_labels_set = train_targets_scored.iloc[validation_features_set.index].copy()

#reset index. We don't really need a index as a number. We can identify our observations by the column 'sig_id'
train_features_set.reset_index(inplace = True)
train_features_set.drop('index', axis = 1, inplace = True)

validation_features_set.reset_index(inplace = True)
validation_features_set.drop('index', axis = 1, inplace = True)

train_labels_set.reset_index(inplace = True)
train_labels_set.drop('index', axis = 1, inplace = True)

validation_labels_set.reset_index(inplace = True)
validation_labels_set.drop('index', axis = 1, inplace = True)


#Remove control observations.
train_features_without_control = train_features_set[train_features_set['cp_type'] != 'ctl_vehicle'].copy()
train_labels_without_control = train_labels_set.iloc[train_features_without_control.index].copy()

train_features_only_control = train_features_set[train_features_set['cp_type'] == 'ctl_vehicle'].copy()

#reset index. 
train_features_without_control.reset_index(inplace = True)
train_features_without_control.drop('index', axis = 1, inplace = True)

train_labels_without_control.reset_index(inplace = True)
train_labels_without_control.drop('index', axis = 1, inplace = True)

train_features_only_control.reset_index(inplace = True)
train_features_only_control.drop('index', axis = 1, inplace = True)

# For the validation and test, we will remove just the column 'cp_type'.
# However, we will not delete the observations. 
# Instead, we will create a separate data frame with 'cp_type' so we can set all MoA equal to zero for these observations.


validation_features_without_control = validation_features_set.drop(['cp_type'], axis = 1).copy()
validation_features_control = validation_features_set[['sig_id', 'cp_type']].copy()

test_features_without_control = test_features.drop(['cp_type'], axis = 1).copy()
test_features_control = test_features[['sig_id', 'cp_type']].copy()

validation_features_only_control = validation_features_set[validation_features_set['cp_type'] == 'ctl_vehicle'].copy()

#These are the same adjustments used before.
train_x = train_features_without_control.drop(['sig_id', 'cp_type'], axis = 1).copy()
train_x_with_control = train_features_set.drop(['sig_id', 'cp_type'], axis = 1).copy()

test_x = validation_features_without_control.drop(['sig_id'], axis = 1).copy()
test_x_control = validation_features_control.copy()
holdout_x = test_features_without_control.drop(['sig_id'], axis = 1).copy()
holdout_x_control = test_features_control.copy()

train_y = train_labels_without_control.drop(['sig_id'], axis = 1).copy()
train_y_with_control = train_labels_set.drop(['sig_id'], axis = 1).copy()
test_y = validation_labels_set.drop(['sig_id'], axis = 1).copy()


train_x['cp_dose'] = train_x['cp_dose'].str.extract(r"([1-2])", expand = True).astype(np.int8)
train_x_with_control['cp_dose'] = train_x_with_control['cp_dose'].str.extract(r"([1-2])", expand = True).astype(np.int8)
test_x['cp_dose'] = test_x['cp_dose'].str.extract(r"([1-2])", expand = True).astype(np.int8)

C:\Users\maxwi\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
def log_loss_mean(y_real, predictions):
    y_real_db = pd.DataFrame(data = y_real)
    
    log_loss_aux = []
    for col in y_real_db:
        log_loss_aux.append(log_loss(y_real_db[col].astype(float), predictions[col].astype(float), labels = [0, 1]))
        
    return np.mean(log_loss_aux)

In [11]:
'''
From: https://github.com/Varal7/ml-tutorial/blob/master/Part2.ipynb
In most ML applications we do mini-batch stochastic gradient descent instead of pure stochastic gradient descent.

Mini-batch SGD is a step between full gradient descent and stochastic gradient descent by computing the average gradient over a small number of examples.

In a nutshell, given n examples:

Full GD: dL/dw = average over all n examples. One step per n examples.
SGD: dL/dw = point estimate over a single example. n steps per n examples.
Mini-batch SGD: dL/dw = average over m << n examples. n / m steps per n examples.
Advantages of mini-batch SGD include a more stable gradient estimate and computational efficiency on modern hardware (exploiting parallelism gives sub-linear to constant time complexity, especially on GPU).

In PyTorch, batched tensors are represented as just another dimension. Most of the deep learning modules assume batched tensors as input (even if the batch size is just 1).

Code from: MITx: 6.86x (Project 3)
'''
def batchify_data(x_data, y_data, batch_size):
    """Takes a set of data points and labels and groups them into batches."""
    # Only take batch_size chunks (i.e. drop the remainder)
    N = int(len(x_data) / batch_size) * batch_size
    batches = []
    for i in range(0, N, batch_size):
        batches.append({
            'x': torch.tensor(x_data[i:i+batch_size], dtype=torch.float32),
            'y': torch.tensor(y_data[i:i+batch_size], dtype=torch.long
        )})
    return batches

In [12]:
def compute_accuracy(predictions, y):
    """Computes the accuracy of predictions against the gold labels, y."""
    predictions_np = predictions.detach().numpy()
    y_np = y.detach().numpy()
    return np.mean(np.equal(predictions_np, y_np))


# Training Procedure
def train_model(train_data, dev_data, model, lr=0.01, momentum=0.9, nesterov = False, n_epochs=30, save_model = 'FNN'):
# def train_model(train_data, dev_data, model, lr=0.01, n_epochs=30):
    """Train a model for N epochs given data and hyper-params."""
    # We optimize with SGD
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=nesterov)
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)  #It takes too long! with epochs = 320, lr = 0.1 and batch_size = 32 it did not help.

    for epoch in range(1, n_epochs):
        print("-------------\nEpoch {}:\n".format(epoch))


        # Run **training***
        train_loss, train_acc, train_out = run_epoch(train_data, model.train(), optimizer)
        print('Train loss: {:.6f} | Train accuracy: {:.6f}'.format(train_loss, train_acc))

        # Run **validation**
        dev_loss, dev_acc, dev_out = run_epoch(dev_data, model.eval(), optimizer)
        print('Val loss:   {:.6f} | Val accuracy:   {:.6f}'.format(dev_loss, dev_acc))
        # Save model
        torch.save(model, save_model + '.pt')
    return 


def run_epoch(data, model, optimizer):
    """Train model for one pass of train data, and return loss, acccuracy"""
    # Gather losses
    losses = []
    batch_accuracies = []

    # If model is in train mode, use optimizer.
    is_training = model.training

    # Iterate through batches
    for batch in tqdm(data):
        # Grab x and y
        x, y = batch['x'], batch['y']
       
        # Get output predictions
        out = model(x)
        y = y.type_as(out)
        
        
        # Predict and store accuracy      
        predictions = torch.round(out)
        batch_accuracies.append(compute_accuracy(predictions, y))


        # Compute loss
        loss = F.binary_cross_entropy(out, y)
        losses.append(loss.data.item())

        # If training, do an update.
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Calculate epoch level scores
    avg_loss = np.mean(losses)
    avg_accuracy = np.mean(batch_accuracies)
    return avg_loss, avg_accuracy, out

In [13]:
#Lets simplify our functions so we do not see the print in each epoch.

# Training Procedure
def train_model_h(train_data, dev_data, model, lr=0.01, momentum=0.9, nesterov = False, n_epochs=30):
    """Train a model for N epochs given data and hyper-params."""
    # We optimize with SGD
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, nesterov=nesterov)
    
    #Try adam
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(1, n_epochs):

        # Run **training***
        train_loss, train_out = run_epoch_h(train_data, model.train(), optimizer)

        # Run **validation**
        dev_loss, dev_out = run_epoch_h(dev_data, model.eval(), optimizer)
        # Save model
        torch.save(model, 'mnist_model_fully_connected.pt')
    return 


def run_epoch_h(data, model, optimizer):
    """Train model for one pass of train data, and return loss, acccuracy"""
    # Gather losses
    losses = []

    # If model is in train mode, use optimizer.
    is_training = model.training

    # Iterate through batches
    for batch in data:
        # Grab x and y
        x, y = batch['x'], batch['y']
       
        # Get output predictions
        out = model(x)
        y = y.type_as(out)      


        # Compute loss
        loss = F.binary_cross_entropy(out, y)
        losses.append(loss.data.item())

        # If training, do an update.
        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Calculate epoch level scores
    avg_loss = np.mean(losses)
    return avg_loss, out

In [14]:
# Lets try differents activation functions
def new_activation_function(parameter):
    if parameter == 0: #ReLu()
        return nn.ReLU()

    if parameter == 1: #tanh()
        return nn.Tanh()

    if parameter == 2: #LeakyReLU()
        return nn.LeakyReLU()
            
    if parameter == 3: #ReLU6()
        return nn.ReLU6() 

    if parameter == 4: #RReLU()
        return nn.RReLU() 

    if parameter == 5: #SELU()
        return nn.SELU()

    if parameter == 6: #CELU()
        return nn.CELU()
 
    if parameter == 7: #GELU()
        return nn.GELU()
       
    if parameter == 8: #Softsign()
        return nn.Softsign()
    
    if parameter == 9: #LogSoftmax()
        return nn.LogSoftmax()

    if parameter == 10: #.Softmax(dim=1)
        return  nn.Softmax(dim=1)

    if parameter == 11: #.ELU()
        return nn.ELU()

    if parameter == 12: #.LogSigmoid()
        return nn.LogSigmoid()
  
    if parameter == 13: #.Sigmoid()
        return nn.Sigmoid()
    
    if parameter == 14: #.PReLU()
        return nn.PReLU()
    
    if parameter == 15: #.Softplus()
        return nn.Softplus()
    
    if parameter == 16: #.Tanhshrink()
        return nn.Tanhshrink() 

Lets first get our benchmark for the log loss.

In [30]:
# Final model 1 hidden layer
start = time.time()

X_train = train_x.to_numpy(copy = True)
X_dev = test_x.to_numpy(copy = True)

Y_train = train_y.to_numpy(copy = True)
Y_dev = test_y.to_numpy(copy = True)

n_epochs = 295               #Hyperparameter
batch_size = 32             #Hyperparameter
 
train_batches = batchify_data(X_train, Y_train, batch_size)
dev_batches = batchify_data(X_dev, Y_dev, batch_size)

train_batches_full_base = torch.tensor(X_train, dtype=torch.float32)
dev_batches_full_base = torch.tensor(X_dev, dtype=torch.float32)


#################################
## Model specification
model = nn.Sequential(
                    nn.Linear(X_train.shape[1], int(0.5*X_train.shape[1])), 
                    nn.ReLU6(),  
                    nn.Linear(int(0.5*X_train.shape[1]), 206),  
                    nn.Sigmoid(),  #So we have probabilities at the end
                )
lr = 0.1 # Hyperparameter
momentum = 0
##################################

#Train model
train_model(train_batches, dev_batches, model, lr = lr, momentum = momentum, nesterov = False, n_epochs = n_epochs, save_model = 'FNN_layer_1')

# Get output.
# Evaluate the model on test data.
out_train = model(train_batches_full_base) 
out_dev = model(dev_batches_full_base)    

out_train = out_train.detach().numpy()
out_train = pd.DataFrame(data = out_train)

# Adjust out_dev for the control group
out_dev = out_dev.detach().numpy()
# Make a matrix with zeros when control.
treatment_flag = test_x_control['cp_type'] == 'trt_cp'
treatment_flag = np.array([treatment_flag])
treatment_flag = np.repeat(treatment_flag.transpose(), 206, axis = 1)
out_dev = out_dev*treatment_flag
out_dev = pd.DataFrame(data = out_dev)
    
print()
print('Log loss train:', log_loss_mean(Y_train, out_train))
print('Log loss dev:', log_loss_mean(Y_dev, out_dev))
# og loss train: 0.013678792980684043
# Log loss dev: 0.015414417115830909


end = time.time()
elapsed = end - start
print()
print('elapsed time:',elapsed)

-------------
Epoch 1:



100%|██████████| 549/549 [00:01<00:00, 460.09it/s]


Train loss: 0.039769 | Train accuracy: 0.993098


100%|██████████| 148/148 [00:00<00:00, 1667.37it/s]


Val loss:   0.021070 | Val accuracy:   0.996826
-------------
Epoch 2:



...

Val loss:   0.015628 | Val accuracy:   0.997281
-------------
Epoch 294:



100%|██████████| 549/549 [00:01<00:00, 514.69it/s]


Train loss: 0.013718 | Train accuracy: 0.997252


100%|██████████| 148/148 [00:00<00:00, 1318.90it/s]


Val loss:   0.015627 | Val accuracy:   0.997281

Log loss train: 0.013678792980684043
Log loss dev: 0.015414417115830909

elapsed time: 352.3526337146759


Considering that there are a visible difference in the distribution of variables between control and treatment, lets run a model considering the control group in the train dataset.

In [31]:
# Final model 1 hidden layer
start = time.time()

X_train = train_x_with_control.to_numpy(copy = True)
X_dev = test_x.to_numpy(copy = True)

Y_train = train_y_with_control.to_numpy(copy = True)
Y_dev = test_y.to_numpy(copy = True)

n_epochs = 295               #Hyperparameter
batch_size = 32             #Hyperparameter
 
train_batches = batchify_data(X_train, Y_train, batch_size)
dev_batches = batchify_data(X_dev, Y_dev, batch_size)

train_batches_full_base = torch.tensor(X_train, dtype=torch.float32)
dev_batches_full_base = torch.tensor(X_dev, dtype=torch.float32)


#################################
## Model specification
model = nn.Sequential(
                    nn.Linear(X_train.shape[1], int(0.5*X_train.shape[1])), 
                    nn.ReLU6(),  
                    nn.Linear(int(0.5*X_train.shape[1]), 206),  
                    nn.Sigmoid(),  #So we have probabilities at the end
                )
lr = 0.1 # Hyperparameter
momentum = 0
##################################

#Train model
train_model(train_batches, dev_batches, model, lr = lr, momentum = momentum, nesterov = False, n_epochs = n_epochs, save_model = 'FNN_layer_1')

# Get output.
# Evaluate the model on test data.
out_train = model(train_batches_full_base) 
out_dev = model(dev_batches_full_base)    

out_train = out_train.detach().numpy()
out_train = pd.DataFrame(data = out_train)

# Adjust out_dev for the control group
out_dev = out_dev.detach().numpy()
# Make a matrix with zeros when control.
treatment_flag = test_x_control['cp_type'] == 'trt_cp'
treatment_flag = np.array([treatment_flag])
treatment_flag = np.repeat(treatment_flag.transpose(), 206, axis = 1)
out_dev = out_dev*treatment_flag
out_dev = pd.DataFrame(data = out_dev)
    
print()
print('Log loss train:', log_loss_mean(Y_train, out_train))
print('Log loss dev:', log_loss_mean(Y_dev, out_dev))
# Log loss train: 0.013678792980684043
# Log loss dev: 0.015414417115830909

end = time.time()
elapsed = end - start
print()
print('elapsed time:',elapsed)

-------------
Epoch 1:



100%|██████████| 595/595 [00:01<00:00, 503.64it/s]


Train loss: 0.036751 | Train accuracy: 0.993622


100%|██████████| 148/148 [00:00<00:00, 1613.01it/s]


Val loss:   0.020806 | Val accuracy:   0.996825
-------------
Epoch 2:



...

Val loss:   0.015603 | Val accuracy:   0.997275
-------------
Epoch 294:



100%|██████████| 595/595 [00:01<00:00, 521.95it/s]


Train loss: 0.012822 | Train accuracy: 0.997462


100%|██████████| 148/148 [00:00<00:00, 1514.25it/s]


Val loss:   0.015602 | Val accuracy:   0.997275

Log loss train: 0.01278403124779123
Log loss dev: 0.015488732493124617

elapsed time: 383.2407178878784


Training with the control group is not a good idea.\
Now lets calculate the mean of the genes and cell variables of the control group and subtract it from the values in the treatment group. 

In [40]:
mean_control_variables = {}
mask_g = train_x.columns.str.contains('^g-*')
train_control_g_variables = train_x.loc[:,mask_g]
train_g_variables = train_x.loc[:,mask_g].copy()
test_g_variables = test_x.loc[:,mask_g].copy()

mask_c = train_x.columns.str.contains('^c-[0-99]')
train_control_c_variables = train_x.loc[:,mask_c]
train_c_variables = train_x.loc[:,mask_c].copy()
test_c_variables = test_x.loc[:,mask_c].copy()

for col in train_control_g_variables:
    mean_control_variables[col] = train_control_g_variables[col].mean()
for col in train_control_c_variables:
    mean_control_variables[col] = train_control_c_variables[col].mean()
    
train_g_variables_subtracted_mean = train_g_variables.copy()
test_g_variables_subtracted_mean = test_g_variables.copy()
for col in train_g_variables:
    train_g_variables_subtracted_mean[col] = train_g_variables[col] - mean_control_variables[col]
    test_g_variables_subtracted_mean[col] = test_g_variables[col] - mean_control_variables[col]
    
train_c_variables_subtracted_mean = train_c_variables.copy()
test_c_variables_subtracted_mean = test_c_variables.copy()
for col in train_c_variables:
    train_c_variables_subtracted_mean[col] = train_c_variables[col] - mean_control_variables[col]
    test_c_variables_subtracted_mean[col] = test_c_variables[col] - mean_control_variables[col]
    
train_x_v2 = train_x[['cp_time','cp_dose']].join(train_g_variables_subtracted_mean)
train_x_v2 = train_x_v2.join(train_c_variables_subtracted_mean)

test_x_v2 = test_x[['cp_time','cp_dose']].join(test_g_variables_subtracted_mean)
test_x_v2 = test_x_v2.join(test_c_variables_subtracted_mean)

In [46]:
# Final model 1 hidden layer
start = time.time()

X_train = train_x_v2.to_numpy(copy = True)
X_dev = test_x_v2.to_numpy(copy = True)

Y_train = train_y.to_numpy(copy = True)
Y_dev = test_y.to_numpy(copy = True)

n_epochs = 295               #Hyperparameter
batch_size = 32             #Hyperparameter
 
train_batches = batchify_data(X_train, Y_train, batch_size)
dev_batches = batchify_data(X_dev, Y_dev, batch_size)

train_batches_full_base = torch.tensor(X_train, dtype=torch.float32)
dev_batches_full_base = torch.tensor(X_dev, dtype=torch.float32)


#################################
## Model specification
model = nn.Sequential(
                    nn.Linear(X_train.shape[1], int(0.5*X_train.shape[1])), 
                    nn.ReLU6(),  
                    nn.Linear(int(0.5*X_train.shape[1]), 206),  
                    nn.Sigmoid(),  #So we have probabilities at the end
                )
lr = 0.1 # Hyperparameter
momentum = 0
##################################

#Train model
train_model(train_batches, dev_batches, model, lr = lr, momentum = momentum, nesterov = False, n_epochs = n_epochs, save_model = 'FNN_layer_1')

# Get output.
# Evaluate the model on test data.
out_train = model(train_batches_full_base) 
out_dev = model(dev_batches_full_base)    

out_train = out_train.detach().numpy()
out_train = pd.DataFrame(data = out_train)

# Adjust out_dev for the control group
out_dev = out_dev.detach().numpy()
# Make a matrix with zeros when control.
treatment_flag = test_x_control['cp_type'] == 'trt_cp'
treatment_flag = np.array([treatment_flag])
treatment_flag = np.repeat(treatment_flag.transpose(), 206, axis = 1)
out_dev = out_dev*treatment_flag
out_dev = pd.DataFrame(data = out_dev)
    
print()
print('Log loss train:', log_loss_mean(Y_train, out_train))
print('Log loss dev:', log_loss_mean(Y_dev, out_dev))
# Log loss train: 0.013678792980684043
# Log loss dev: 0.015414417115830909

end = time.time()
elapsed = end - start
print()
print('elapsed time:',elapsed)

-------------
Epoch 1:



100%|██████████| 549/549 [00:01<00:00, 510.64it/s]


Train loss: 0.039827 | Train accuracy: 0.993030


100%|██████████| 148/148 [00:00<00:00, 1698.85it/s]


Val loss:   0.021353 | Val accuracy:   0.996808
-------------
Epoch 2:



...

Val loss:   0.015663 | Val accuracy:   0.997287
-------------
Epoch 294:



100%|██████████| 549/549 [00:01<00:00, 505.95it/s]


Train loss: 0.013714 | Train accuracy: 0.997256


100%|██████████| 148/148 [00:00<00:00, 1187.18it/s]


Val loss:   0.015662 | Val accuracy:   0.997286

Log loss train: 0.0136750302720019
Log loss dev: 0.015448702754989111

elapsed time: 352.56719756126404


This did not work.\
We saw that the genes and cell variables have a different change in distribution when considering the other variables in our dataset. Therefore, lets model first only with the genes variables and after only with the cell varialbes.

In [47]:
train_x_g = train_x[['cp_time','cp_dose']].join(train_g_variables).copy()
test_x_g = test_x[['cp_time','cp_dose']].join(test_g_variables).copy()

# Final model 1 hidden layer
start = time.time()

X_train = train_x_g.to_numpy(copy = True)
X_dev = test_x_g.to_numpy(copy = True)

Y_train = train_y.to_numpy(copy = True)
Y_dev = test_y.to_numpy(copy = True)

n_epochs = 295               #Hyperparameter
batch_size = 32             #Hyperparameter
 
train_batches = batchify_data(X_train, Y_train, batch_size)
dev_batches = batchify_data(X_dev, Y_dev, batch_size)

train_batches_full_base = torch.tensor(X_train, dtype=torch.float32)
dev_batches_full_base = torch.tensor(X_dev, dtype=torch.float32)


#################################
## Model specification
model = nn.Sequential(
                    nn.Linear(X_train.shape[1], int(0.5*X_train.shape[1])), 
                    nn.ReLU6(),  
                    nn.Linear(int(0.5*X_train.shape[1]), 206),  
                    nn.Sigmoid(),  #So we have probabilities at the end
                )
lr = 0.1 # Hyperparameter
momentum = 0
##################################

#Train model
train_model(train_batches, dev_batches, model, lr = lr, momentum = momentum, nesterov = False, n_epochs = n_epochs, save_model = 'FNN_layer_1')

# Get output.
# Evaluate the model on test data.
out_train = model(train_batches_full_base) 
out_dev = model(dev_batches_full_base)    

out_train = out_train.detach().numpy()
out_train = pd.DataFrame(data = out_train)

# Adjust out_dev for the control group
out_dev = out_dev.detach().numpy()
# Make a matrix with zeros when control.
treatment_flag = test_x_control['cp_type'] == 'trt_cp'
treatment_flag = np.array([treatment_flag])
treatment_flag = np.repeat(treatment_flag.transpose(), 206, axis = 1)
out_dev = out_dev*treatment_flag
out_dev = pd.DataFrame(data = out_dev)
    
print()
print('Log loss train:', log_loss_mean(Y_train, out_train))
print('Log loss dev:', log_loss_mean(Y_dev, out_dev))

out_train_g = out_train.copy()
out_dev_g = out_dev.copy()

# Log loss train: 0.013678792980684043
# Log loss dev: 0.015414417115830909

end = time.time()
elapsed = end - start
print()
print('elapsed time:',elapsed)

-------------
Epoch 1:



100%|██████████| 549/549 [00:00<00:00, 566.41it/s]


Train loss: 0.039568 | Train accuracy: 0.993081


100%|██████████| 148/148 [00:00<00:00, 1091.15it/s]


Val loss:   0.020855 | Val accuracy:   0.996810
-------------
Epoch 2:



...

Val loss:   0.016004 | Val accuracy:   0.997234
-------------
Epoch 294:



100%|██████████| 549/549 [00:01<00:00, 482.81it/s]


Train loss: 0.014437 | Train accuracy: 0.997188


100%|██████████| 148/148 [00:00<00:00, 1592.40it/s]


Val loss:   0.016004 | Val accuracy:   0.997234

Log loss train: 0.01439856950588105
Log loss dev: 0.015764893184396462

elapsed time: 326.192054271698


In [52]:
train_x_c = train_x[['cp_time','cp_dose']].join(train_c_variables).copy()
train_x_c = train_x_c.join(out_train_g)
test_x_c = test_x[['cp_time','cp_dose']].join(test_c_variables).copy()
test_x_c = test_x_c.join(out_dev_g)

# Final model 1 hidden layer
start = time.time()

X_train = train_x_c.to_numpy(copy = True)
X_dev = test_x_c.to_numpy(copy = True)

Y_train = train_y.to_numpy(copy = True)
Y_dev = test_y.to_numpy(copy = True)

n_epochs = 350               #Hyperparameter
batch_size = 32             #Hyperparameter
 
train_batches = batchify_data(X_train, Y_train, batch_size)
dev_batches = batchify_data(X_dev, Y_dev, batch_size)

train_batches_full_base = torch.tensor(X_train, dtype=torch.float32)
dev_batches_full_base = torch.tensor(X_dev, dtype=torch.float32)


#################################
## Model specification
model = nn.Sequential(
                    nn.Linear(X_train.shape[1], int(0.5*X_train.shape[1])), 
                    nn.ReLU6(),  
                    nn.Linear(int(0.5*X_train.shape[1]), 206),  
                    nn.Sigmoid(),  #So we have probabilities at the end
                )
lr = 0.1 # Hyperparameter
momentum = 0
##################################

#Train model
train_model(train_batches, dev_batches, model, lr = lr, momentum = momentum, nesterov = False, n_epochs = n_epochs, save_model = 'FNN_layer_1')

# Get output.
# Evaluate the model on test data.
out_train = model(train_batches_full_base) 
out_dev = model(dev_batches_full_base)    

out_train = out_train.detach().numpy()
out_train = pd.DataFrame(data = out_train)

# Adjust out_dev for the control group
out_dev = out_dev.detach().numpy()
# Make a matrix with zeros when control.
treatment_flag = test_x_control['cp_type'] == 'trt_cp'
treatment_flag = np.array([treatment_flag])
treatment_flag = np.repeat(treatment_flag.transpose(), 206, axis = 1)
out_dev = out_dev*treatment_flag
out_dev = pd.DataFrame(data = out_dev)
    
print()
print('Log loss train:', log_loss_mean(Y_train, out_train))
print('Log loss dev:', log_loss_mean(Y_dev, out_dev))

out_train_c = out_train.copy()
out_dev_c = out_dev.copy()

# Log loss train: 0.013678792980684043
# Log loss dev: 0.015414417115830909

end = time.time()
elapsed = end - start
print()
print('elapsed time:',elapsed)

-------------
Epoch 1:



100%|██████████| 549/549 [00:00<00:00, 955.65it/s]


Train loss: 0.046260 | Train accuracy: 0.990487


100%|██████████| 148/148 [00:00<00:00, 1902.49it/s]


Val loss:   0.020841 | Val accuracy:   0.996625
-------------
Epoch 2:



...

Val loss:   0.017035 | Val accuracy:   0.997070
-------------
Epoch 349:



100%|██████████| 549/549 [00:00<00:00, 1067.70it/s]


Train loss: 0.017852 | Train accuracy: 0.996894


100%|██████████| 148/148 [00:00<00:00, 1908.07it/s]


Val loss:   0.017033 | Val accuracy:   0.997070

Log loss train: 0.017827467628370744
Log loss dev: 0.016800863957196283

elapsed time: 249.29914450645447


It looks like the best model is with only the variables in the train dataset without any modification.